In [1]:
import math
import itertools

#### To make lazy evaluation work for this polygon, I introduced two trackers, one tracker `object.__flag` for tracking if the independent parameters have been modified using setter methods, and the other tracker `object.__repeats` for tracking if dependent parameters are called multiple times. Both the tracker parameters are defined as mutable list containers because we need to modify per requirements. However as they are defined using `__` so user cannot modify them accidentally or intentionally.

Note: Only independent parameters are defined with setter methods (These are num_edges and circum_radius). Hence these independent parameters are defined as a regular variable without any `_`. Rest of the parameters have no setters associated with them, and are defined using `_`.

Exception: I could have made all the dependent parameters with `__` to stop any accidental or intentional editing of parameters but I go with the python standard notation of single `_`. As a result, changes to dependent parameters is possible and could give 'incorrect' results with no warnings or error messages which is dangerous. One example is shown to illustrate this result.

In [2]:
# Regular Convex Polygon Object

class Polygon:
    '''
    This Polygon takes 2 parameters -- number of edges (N) and circum-radius (R), and the related parameters
    such as vertices, interior angle, edge length, apothem, area and perimeter are obtained using get methods.
    Access to modify by name operator is available only for num_edges and circum_rad, and any such 
    modifications will reflect accordingly in the dependent parameters accessed by get_{attribute} methods.
    '''
    
    def __init__(self, num_edges=3, circum_rad=6):
        if not isinstance(num_edges, int):
            raise TypeError(f'"num_edges" is an integer; TypeError')
        if num_edges < 3:
            raise ValueError(f'Polygon has a minimum of 3 edges; ValueError')
        self.num_edges = num_edges
        self.circum_rad = circum_rad
        self._vertices = None
        self._int_angle = None
        self._edge_length = None
        self._apothem = None
        self._area = None
        self._perimeter = None
        self.__flag = [0, 0] # changes when setter for num_edges and/or circum_radius is called
        self.__repeats = [0, 0, 0, 0, 0, 0] # to track number of times methods get called for 

    @property
    def get_num_edges(self):
        return self.num_edges
    
    @property
    def get_circum_rad(self):
        return self.circum_rad
    
    # Allowing num_edges and circum_radius to be modified without checking if they have existing values...
    # Because num_edges and circum_radius are independent parameters on which rest of other parameters depend
    
    @get_num_edges.setter
    def set_num_edges(self, n):
        print('setter for num_edges called -- you are modifying the original num_edges:')
        self.num_edges = n
        if self.__flag[0] == 1:
            self.__flag[0] = 0
        self.__flag[0] = 1
        self.__repeats = [0, 0, 0, 0, 0, 0]

    @get_circum_rad.setter
    def set_circum_rad(self, r):
        print('setter for circum_radius called -- you are modifying original circum_radius:')
        self.circum_rad = r
        if self.__flag[1] == 1:
            self.__flag[1] = 0
        self.__flag[1] = 1
        self.__repeats = [0, 0, 0, 0, 0, 0]

    # Setters for dependent properties not allowed
    
    @property
    def vertices(self):
        # print('getter for vertices')
        cnts = self.__repeats[0]
        if cnts == 0:
            if self._vertices is None or sum(self.__flag) >= 1:
                # print(f'executing first time ...')
                # return self.get_num_edges
                self._vertices = self.get_num_edges
                self.__repeats[0] = 1
        return self._vertices
        
    @property
    def int_angle(self):
        # print('getter for interior angle')
        cnts = self.__repeats[1]
        if cnts == 0:
            if self._int_angle is None or sum(self.__flag) >= 1:
                # print(f'executing first time ...')
                self._int_angle = (self.get_num_edges - 2) * 180 / self.get_num_edges
                self.__repeats[1] = 1
        return self._int_angle

    @property
    def edge_length(self):
        # print('getter for edge_length')
        cnts = self.__repeats[2]
        if cnts == 0:
            if self._edge_length is None or sum(self.__flag) >= 1:
                # print(f'executing first time ...')
                self._edge_length = 2*self.get_circum_rad * math.sin(math.pi / self.get_num_edges)
                self.__repeats[2] = 1
        return self._edge_length

    @property
    def apothem(self):
        # print('getter for apothem')
        cnts = self.__repeats[3]
        if cnts == 0:
            if self._apothem is None or sum(self.__flag) >= 1:
                # print(f'executing first time ...')
                self._apothem = self.get_circum_rad * math.cos(math.pi / self.get_num_edges)
                self.__repeats[3] = 1
        return self._apothem

    @property
    def area(self):
        # print('getter for area')
        cnts = self.__repeats[4]
        if cnts == 0:
            if self._area is None or sum(self.__flag) >= 1:
                # print(f'executing first time ...')
                self._area = self.get_num_edges * self.edge_length * self.apothem / 2
                self.__repeats[4] = 1
        return self._area

    @property
    def perimeter(self):
        # print('getter for perimeter')
        cnts = self.__repeats[5]
        if cnts == 0:
            if self._perimeter is None or sum(self.__flag) >= 1:
                # print(f'executing first time ...')
                self._perimeter = self.get_num_edges * self.edge_length
                self.__repeats[5] = 1
        return self._perimeter

    def __repr__(self):
        return f'Polygon(edges={self.get_num_edges}, rad={self.get_circum_rad})'

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            if self.vertices == other.vertices and self.get_circum_rad == other.get_circum_rad:
                return True
            else:
                return False
        else:
            return print(f'Compare instances of same class: Right-side instance not a Polygon')

    def __gt__(self, other):
        if isinstance(other, self.__class__):
            if self.vertices > other.vertices:
                return True
            else:
                return False
        else:
            return print(f'Compare instances of same class: Right-side instance not a Polygon')

In [3]:
def test_polygon():
    abs_tol = 0.001
    rel_tol = 0.001
    
    try:
        p = Polygon(2, 10)
        assert False, ('Creating a Polygon with 2 sides: '
                       ' Exception expected, not received')
    except ValueError:
        pass
                       
    n = 3
    R = 1
    p = Polygon(n, R)
    assert str(p) == 'Polygon(edges=3, rad=1)', f'actual: {str(p)}'
    assert p.vertices == n, (f'actual: {p.vertices},'
                                   f' expected: {n}')
    assert p.get_num_edges == n, f'actual: {p.get_num_edges}, expected: {n}'
    assert p.get_circum_rad == R, f'actual: {p.get_circum_rad}, expected: {R}'
    assert p.int_angle == 60, (f'actual: {p.int_angle},'
                                    ' expected: 60')
    n = 4
    R = 1
    p = Polygon(n, R)
    assert p.int_angle == 90, (f'actual: {p.int_angle}, '
                                    ' expected: 90')
    assert math.isclose(p.area, 2, 
                        rel_tol=abs_tol, 
                        abs_tol=abs_tol), (f'actual: {p.area},'
                                           ' expected: 2.0')
    
    assert math.isclose(p.edge_length, math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.edge_length},'
                                          f' expected: {math.sqrt(2)}')
    
    assert math.isclose(p.perimeter, 4 * math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.perimeter},'
                                          f' expected: {4 * math.sqrt(2)}')
    
    assert math.isclose(p.apothem, 0.707,
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.apothem},'
                                          ' expected: 0.707')
    p = Polygon(6, 2)
    assert math.isclose(p.edge_length, 2,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.apothem, 1.73205,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.area, 10.3923,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.perimeter, 12,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.int_angle, 120,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p = Polygon(12, 3)
    assert math.isclose(p.edge_length, 1.55291,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.apothem, 2.89778,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.area, 27,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.perimeter, 18.635,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.int_angle, 150,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p1 = Polygon(3, 10)
    p2 = Polygon(10, 10)
    p3 = Polygon(15, 10)
    p4 = Polygon(15, 100)
    p5 = Polygon(15, 100)
    
    assert p2 > p1
    assert p2 < p3
    assert p3 != p4
    assert p1 != p4
    assert p4 == p5

In [4]:
test_polygon()

In [5]:
p = Polygon(5, 3)

In [6]:
p

Polygon(edges=5, rad=3)

In [7]:
p.vertices, p.int_angle, p.edge_length, p.apothem, p.area, p.perimeter

(5,
 108.0,
 3.526711513754839,
 2.4270509831248424,
 21.398771616640957,
 17.633557568774194)

In [8]:
p.set_circum_rad = 5

setter for circum_radius called -- you are modifying original circum_radius:


In [9]:
p

Polygon(edges=5, rad=5)

In [10]:
p.vertices, p.int_angle, p.edge_length, p.apothem, p.area, p.perimeter

(5,
 108.0,
 5.877852522924732,
 4.045084971874737,
 59.44103226844711,
 29.38926261462366)

In [11]:
# Unintended usage of `_` defined variables. Now that we change the value like this, it still
# prints out the last saved output that doesnt reflect the change in the parameter value.

p._vertices = 9

Notice that only the number of vertices got changed, and the values of the rest of the properties (dependent properties) don't reflect the appropriate change.

In [12]:
p.vertices, p.int_angle, p.edge_length, p.apothem, p.area, p.perimeter

(9,
 108.0,
 5.877852522924732,
 4.045084971874737,
 59.44103226844711,
 29.38926261462366)

In [13]:
class PolygonIterator:
    '''
    This class takes in number of vertices for the largest polygon in the sequence. Currently the sequence type is chosen as list,
    but it can be changed later, if required. The SequencePolygon class also takes in a circum_radius and it is assumed to be common
    for all of the polygons in the sequence.
    '''
    def __init__(self, num_vertices=3, circum_rad=6):
        if not isinstance(num_vertices, int):
            raise TypeError(f'"num_vertices" is an integer; TypeError')
        if num_vertices < 3:
            raise ValueError(f'Polygon has a minimum of 3 vertices; ValueError')
        
        self.num_vertices = num_vertices
        self.circum_rad = circum_rad
        # self._pgon_pairs = ((Polygon, (edges, self.circum_rad)) for edges in range(3, self.num_vertices+1))
        self.pairs = ((edges, self.circum_rad) for edges in range(3, self.num_vertices+1))
        self.__start = 0
        
    def __len__(self):
        return self.num_vertices - 2
    
    def __repr__(self):
        return f'PolygonIterator(iterable: edges={self.num_vertices}, fixed: rad={self.circum_rad})'
    
    def __iter__(self):
        return self.PgonIter(num_polygons=len(self), polypairs = self.pairs)
    
    def start_again(self):
        self.pairs = ((edges, self.circum_rad) for edges in range(3, self.num_vertices+1))
        return self.pairs       
                
    def __getitem__(self, index):
        self.pairs = self.start_again()
        try:
            if index not in range(self.num_vertices - 2):
                raise IndexError
        except Exception:
            print(f'IndexError: Index out of range')
            return f''
        else:
            keyval_pairs = dict(enumerate(self.pairs))
            if index >= len(self):
                raise IndexError
            vals = keyval_pairs[index]
            edges = vals[0]
            circum_radius = vals[1]
            self.pairs = self.start_again()
            return Polygon(edges, circum_radius)
            
    class PgonIter:
        def __init__(self, num_polygons, polypairs):
            self.length = num_polygons
            self.i = 0
            self.polyps = polypairs
            
        def __iter__(self):
            return self
        
        def __next__(self):
            # print(f'Executing PgonIter __next__')
            try:
                if self.i >= self.length:
                    raise StopIteration
            except StopIteration:
                print(f'StopIteration: Iterator reached max_value')
                return print(f'')
            else:
                current = self.i
                self.i += 1
                res = next(self.polyps)
                return Polygon(*res)

In [14]:
s = PolygonIterator(5, 5)

In [15]:
s[1]

Polygon(edges=4, rad=5)

In [16]:
next(iter(s))

Polygon(edges=3, rad=5)

In [17]:
item = iter(s)

In [18]:
next(item)

Polygon(edges=4, rad=5)

In [19]:
next(iter(s))

Polygon(edges=5, rad=5)

In [20]:
# Test cases for the PolygonIterator

def test_polygon_iterator():
    abs_tol = 0.001
    rel_tol = 0.001
    
    try:
        p = PolygonIterator(2, 10)
        assert False, ('Creating a Polygon with 2 sides: '
                       ' Exception expected, not received')
    except ValueError:
        pass
                       
    n = 3
    R = 1
    p = PolygonIterator(n, R)
    assert str(p) == 'PolygonIterator(iterable: edges=3, fixed: rad=1)', f'actual: {str(p)}'
    assert len(p) == n-3+1, f'actual: {len(p)}'
    try:
        print(p[1])
        assert False, ('Printing out of range item: '
                       'IndexError exception expected, not received')
    except AssertionError:
        pass
    
    # Now we are going to access individual Polygon using __getitem__() i.e., dictionary key-value of object
    # if p is the PolygonIterator instance, p[0] denotes Polygon(3,R), p[1] = Polygon(4,R), etc. for any R.
    
    n = 4
    R = 1
    p = PolygonIterator(n, R)
    # Below I used the index 1 (p[1]) as p will generate two polygons Polygon(3, 1) and Polygon(4, 1)
    assert p[1].int_angle == 90, (f'actual: {p[1].int_angle}, '
                                    ' expected: 90')
    assert math.isclose(p[1].area, 2, 
                        rel_tol=abs_tol, 
                        abs_tol=abs_tol), (f'actual: {p[1].area},'
                                           ' expected: 2.0')
    
    assert math.isclose(p[1].edge_length, math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p[1].edge_length},'
                                          f' expected: {math.sqrt(2)}')
    
    assert math.isclose(p[1].perimeter, 4 * math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p[1].perimeter},'
                                          f' expected: {4 * math.sqrt(2)}')
    
    assert math.isclose(p[1].apothem, 0.707,
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p[1].apothem},'
                                          ' expected: 0.707')
    
    p = PolygonIterator(6, 2)
    # Here p[3] refers to Polygon(6, 2) as p will create a sequence of Polygons i.e., Polygon(3, 2), Polygon(4, 2), Polygon(5, 2) and Polygon(6, 2)
    assert math.isclose(p[3].edge_length, 2,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[3].apothem, 1.73205,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[3].area, 10.3923,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[3].perimeter, 12,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[3].int_angle, 120,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p = PolygonIterator(12, 3)
    assert math.isclose(p[9].edge_length, 1.55291,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[9].apothem, 2.89778,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[9].area, 27,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[9].perimeter, 18.635,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p[9].int_angle, 150,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p1 = PolygonIterator(3, 10)
    p2 = PolygonIterator(10, 10)
    p3 = PolygonIterator(15, 10)
    p4 = PolygonIterator(15, 100)
    p5 = PolygonIterator(15, 100)
    
    assert all([p2[i] > p1[0] for i in range(1, 8)])
    assert all([p2[7] < p3[i] for i in range(8, 13)])
    assert all([p3[i] != p4[i] for i in range(13)])
    assert p1[0] != p4[0]
    assert all([p4[i] == p5[i] for i in range(13)])

In [21]:
test_polygon_iterator()

IndexError: Index out of range

